In [1]:
import sys 
sys.path.append('..')

In [2]:
import os
from src.datasets.mimic_cxr_dataset import MIMICCXRDataModule
from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
seed_everything(42, workers=True)


/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

augmentations = {'train':
    T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        T.RandomGrayscale(p=0.2),
        T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ]),
    'val':
    T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        T.RandomGrayscale(p=0.2),
        T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ])
}


In [4]:
data_path = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
datamodule = MIMICCXRDataModule(data_path, transforms=augmentations, only_images=False, batch_size=32,
                                limit_num_samples=128)
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()

100%|██████████| 1/1 [00:00<00:00,  1.50it/s]


In [5]:
for batch in train_loader:
    print("Image shape : ",batch["image"].shape)
    print(batch["text"][0])# First sample in the batch
    print("Input Ids shape : ",batch["input_ids"].shape)
    print("Targets shape : ",batch["targets"].shape)
    break

Image shape :  torch.Size([32, 3, 224, 224])
<BOS> <image> Output: FINAL REPORT INDICATION:  Back pain and cough.  Evaluate for pneumonia. COMPARISONS:  None. TECHNIQUE:  PA and lateral views of the chest were obtained. FINDINGS:  The lungs are clear without consolidation or edema.  There is no pleural effusion or pneumothorax.  The cardiomediastinal silhouette is normal. The osseous structures are unremarkable. IMPRESSION:  No acute cardiopulmonary process. <EOC>
Input Ids shape :  torch.Size([32, 1, 256])
Targets shape :  torch.Size([32, 1, 256])


In [6]:
from src.models.multimodal.flamingo_palm_clip import FlamingoClipPalm
import pytorch_lightning as pl

In [7]:
pretrained_clip_path = '/Users/caghankoksal/Desktop/development/PubMedCLIP_ViT32.pth'

In [8]:
model = FlamingoClipPalm(pretrained_clip_path = pretrained_clip_path)

In [9]:
trainer = pl.Trainer(max_epochs=200,deterministic=True,
                     accelerator="gpu", devices=1)
trainer.fit(model, train_dataloaders=train_loader,)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name          | Type              | Params
----------------------------------------------------
0 | vit_clip      | VisionTransformer | 87.8 M
1 | flamingo_palm | FlamingoPaLM      | 167 M 
----------------------------------------------------
79.7 M    Trainable params
87.8 M    Non-trainable params
167 M     Total params
670.105   Total estimated model params size (MB)
/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: Possib

Epoch 2:   0%|          | 0/4 [00:00<?, ?it/s, loss=9.59, v_num=1]        

/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
